In [1]:
import pandas as pd
import numpy as np
import PyPDF2
import string
import re
import os
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from nltk.corpus import stopwords
from nltk import word_tokenize
pd.set_option('display.max_columns', 100)

In [2]:
with open('dictionary.pickle', 'rb') as handle:
    my_dict = pickle.load(handle)
Cases = pd.read_csv('data/Cases2018.csv', encoding='iso-8859-1')

In [3]:
Labels = Cases[['docket','partyWinning']]
Text = pd.DataFrame.from_dict(my_dict, orient="index")

In [4]:
Text = Text.rename(index=str, columns={0: "Document"})

In [5]:
Text['docket'] = Text.index
Text.reset_index(drop=True)

,Document,docket
0,1 IN THE SU...,00-1011
1,12\n3\n4\n5\n6\n7\n8\n910\n11\n12\n13\n14\n15\...,00-1021
2,1 2 \n ...,00-1045
3,12345678910\n111213141516171819202122232425 IN...,00-10666
4,12\n3\n4\n5\n6\n7\n8\n910\n11\n12\n13\n14\n15\...,00-1072
5,12\n3\n4\n5\n6\n7\n8\n910\n11\n12\n13\n14\n15\...,00-1073
6,1234\n5\n678\n910\n11\n12\n131415\n161718\n19\...,00-1089
7,12\n3\n4\n5\n6\n7\n8\n910\n11\n12\n13\n14\n15\...,00-1167
8,12\n3\n4\n5\n6\n7\n8\n910\n11\n12\n13\n14\n15\...,00-1187
9,1 IN THE SU...,00-121


In [6]:
Text = pd.merge(Text, Labels, on='docket', how='inner')
Text.drop(Text[Text.partyWinning==2.0].index, inplace = True)
Labels = Text[['partyWinning']]
Text.drop('partyWinning', axis=1, inplace= True)

In [11]:
Text.iloc[952]

Document    \n   ˆ\n#$ & ",        \n \n \n   \n\n\n\n\n\n...
docket                                                12-8561
Name: 953, dtype: object

In [13]:
my_dict['12-7822']

'   ˙\n%\' # $,        \n \n   \n\n\n\n\n\n\n\n   ˙% \' #$ ,     \n \n \n \n \n \n \n \n \n    \n˙ %\'#$\n,         \n  \n  \n        ˙ % \n\'\n# $ \n, \n  \n \n  \n \n \n  \n \n  \n \n \n \n ˙   \n  ˙\n%\n\'\n#\n$ , \n \n \n  \n\n \n \n   \n   %   \n  \n˙ % \n\' # \n$ ,  \n \n  \n \n \n \n       \'      ˙ % \n\' # $\n, \n \n  \n  \n\n \n \n \n \n   \n \n#   \n  ˙ %\n\'\n# $ \n, \n  \n   \n \n \n \n \n \n \n $      \n˙ \n%\n\' \n# $\n,  \n \n  \n    \n   \n   ,     ˙ % \' \n# \n$ , \n  \n \n \n \n\n \n    \n \n \n\n \n      \n ˙ % \' #$ \n,  \n  \n  \n \n \n      \n     \n \n ˙ % \' # \n$ ,\n\n \n     \n \n \n  \n \n \n      \n ˙ \n% \' # $ ,  \n \n  \n   \n    \n      \n  \n˙ % \'# $ , \n  \n    \n \n   \n \n \n \n       \n ˙ % \' \n# $ ,       \n  \n\n \n  \n \n     \n \n ˙ %\' # $ ,\n \n\n \n \n  \n    \n \n \n     \n \n ˙ % \' # $ ,  \n     \n \n \n     \n     \n  ˙ %\n\' \n# $\n,   \n   \n \n   \n            ˙% \' # \n$ ,     \n   \n  \n \n  \n        \n ˙ \n%\n\' # $\n,          

In [20]:
def convert_pdf_to_txt(path_to_file):
    rsrcmgr = PDFResourceManager()
    retstr = io.StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path_to_file, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [22]:
test  = convert_pdf_to_txt('ArgText/12-7822.pdf')

In [18]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import HTMLConverter,TextConverter,XMLConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import io

In [24]:
def preprocess2(data):
    reviews_tokens = []
    for review in data:
        review = review.lower() #Convert to lower-case words
        raw_word_tokens = re.findall(r'(?:\w+)', review,flags = re.UNICODE) #remove pontuaction
        word_tokens = [w for w in raw_word_tokens if w not in stoplist and not w.isdigit()] # do not add stop words
        reviews_tokens.append(word_tokens)
    return reviews_tokens #return all tokens

[[],
 ['f'],
 ['f'],
 [],
 ['c'],
 [],
 [],
 ['l'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['h'],
 [],
 [],
 [],
 ['u'],
 ['p'],
 ['r'],
 [],
 [],
 [],
 [],
 ['c'],
 [],
 ['u'],
 ['r'],
 [],
 [],
 [],
 ['f'],
 [],
 [],
 ['h'],
 [],
 [],
 ['u'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['l'],
 [],
 [],
 ['r'],
 [],
 ['f'],
 [],
 ['r'],
 [],
 [],
 [],
 [],
 [],
 ['z'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['p'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['r'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [

In [27]:
user_defined_stop_words = ['-x','--','x•','n','w','e','washington','x','v','alderson','petitioner','respondent','reporting','company','supreme','court','united','states','states•','•v','•petitioner'] 

i = stopwords.words('english')
j = list(string.punctuation) + user_defined_stop_words

stoplist = set(i).union(j)